<a href="https://colab.research.google.com/github/hseongeun/projectmanager/blob/main/2_%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%B6%84%EC%84%9D_%EC%98%88%EC%A0%9C_%EB%B9%84%EC%A0%95%ED%98%95%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%B9%B4%EC%B9%B4%EC%98%A4%ED%86%A1_%EB%8C%80%ED%99%94%EB%B0%A9_%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%B6%84%EC%84%9D_%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [비정형 데이터 -> 정형 데이터로 만들기] : 카카오톡 데이터 분석

- [참고] https://wikidocs.net/141389

## 0. 데이터 준비하기(모바일에서)

1. 분석할 특정 채팅방 선택하기
2. 화면 우측 상단 더보기(줄세개) 선택하기
3. 화면 우측 하단 채팅방 설정(톱니바퀴) 선택하기
4. 대화 내용 내보내기 선택 (텍스트 메시지만 보내기)
5. 메일 등을 이용하여 다운로드 받아 data 폴더로 이동시키기

## 1. 원본데이터 살펴보기

In [11]:
import pandas as pd

# 자신의 카카오톡 데이터 파일명으로 변경한다.
file = './sample_data/KaKaoTalkChats.txt'
df = pd.read_table(file)
df

,더영이 님과 카카오톡 대화
0,저장한 날짜 : 2023-09-05 10:26:09
1,--------------- 2022년 7월 17일 일요일 ---------------
2,[홍성은] [오후 7:29] 키키 조아조아 ~~
3,[홍성은] [오후 7:30] 카페도 가자💛💛
4,[더영이] [오후 11:43] 얌 당여나지 ~~!!
...,...
2864,[홍성은] [오전 9:44] ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ나도 ㅠㅠㅠㅠ
2865,[홍성은] [오전 9:44] 아니 우리
2866,[홍성은] [오전 9:44] 왜 솔로야
2867,[홍성은] [오전 9:44] 언제까지 솔로야


## 2. 데이터 전처리(카카오톡 데이터 파싱하기)
- 참고:  https://regexr.com/

[예제 문장] 2023년 3월 22일 오전 11:36, 송혜경 : 이모티콘 (Android)

1. 카카오톡 데이터 쪼개기
    - 날짜와 시간: (iOS: . / Android: 년,월,일
    - 보낸 사람과 텍스트
2. 카카오톡 데이터 파싱하기
    - 정규표현식 매칭 & 메시지 파싱
3. 데이터 프레임으로 만들기

In [33]:
import re
import pandas as pd
import os


def katalk_msg_parse(file_path):
    my_katalk_data = list()

    # 정규 패턴 정의
    #katalk_msg_pattern = "[0-9]{4}[년.] [0-9]{1,2}[월.] [0-9]{1,2}[일.] 오\S [0-9]{1,2}:[0-9]{1,2},.*:"
    katalk_msg_pattern = "\[.*\] \[오\S [0-9]{1,2}:[0-9]{1,2}\] "
    date_info = "[0-9]{4}년 [0-9]{1,2}월 [0-9]{1,2}일 \S요일"
    in_out_info = "\[.*\] \[오\S [0-9]{1,2}:[0-9]{1,2}\] "


    # 파일 데이터 읽기
    cnt = 0
    f = open(file_path, encoding='utf-8')
    for line in f :
        cnt += 1
        if re.match(date_info, line) or re.match(in_out_info, line):
            continue
        elif line == '\n': # 엔터
            continue
        elif re.match(katalk_msg_pattern, line):  # 카카오톡 메시지 패턴
            print(line)
            line = line.split(" ")
            user_name = line[0]
            user_text = line[1].split(" ", maxsplit = 1)
            date_time = date_info


            # date_time = line[0]                           # 년월일 시간
            # user_text = line[1].split(" : ", maxsplit=1)
            # user_name = user_text[0].strip()              # 메시지 보낸 사람
            # text = user_text[1].strip()                   # 메시지
            # # DataFrame으로 나타내기 위해 my_katalk_data 리스트에 딕셔너리 타입으로 추가하기
            # my_katalk_data.append({'date_time': date_time,
            #                        'user_name': user_name,
            #                        'text': text
            #                        })

        else:
            if len(my_katalk_data) > 0:
                my_katalk_data[-1]['text'] += "\n"+line.strip()
    print(f'파일 read count: {cnt}')

    # DataFrame 데이터로 만들기
    my_katalk_df = pd.DataFrame(my_katalk_data)
    print(f'DataFrame count: {len(my_katalk_df)}')
    return my_katalk_df


df = katalk_msg_parse(file)

print(df)

파일 read count: 2871
DataFrame count: 0
Empty DataFrame
Columns: []
Index: []


## 3. 데이터 전처리(컬럼 분할)

### 컬럼 추가하기

In [30]:
df['datetime'] = df['date_time']

KeyError: ignored

### 컬럼 순서 변경하기

In [14]:
df.columns
df = df[ ['date_time', 'datetime', 'user_name', 'text'] ]

KeyError: ignored

### 날짜, 시간 세분화하기

In [16]:
# 오전/오후 --> AM/PM 으로 변경
df['datetime'] = df['datetime'].str.replace('오전', 'AM')
df['datetime'] = df['datetime'].str.replace('오후', 'PM')
df.head(3)

KeyError: ignored

In [ ]:
# pandas 표준 date_time 포맷으로 변경
# df['datetime'] = pd.to_datetime(df['datetime'], format='%Y. %m. %d. %p %I:%M')
df['datetime'] = pd.to_datetime(df['datetime'], format='%Y년 %m월 %d일 %p %H:%M')
df.head(3)

In [ ]:
# text	date	year	month	day	weekday	hour 컬럼 추가
df['date'] = df['datetime'].dt.date
df['year'] = df['datetime'].dt.year
df['month'] = df['datetime'].dt.month
df['day'] = df['datetime'].dt.day
df['weekday'] = df['datetime'].dt.day_name()
df['hour'] = df['datetime'].dt.hour
df.head(3)

### 카카오톡 메시지 길이

In [17]:
# 메시지 길이, 메시지 단어 개수  컬럼 추가
df['msg_len'] = df['text'].str.len()  # 메시지 길이
df['msg_word_count'] = df['text'].str.split().str.len() #메시지 단어 개수
df.head()

KeyError: ignored

### 사진, 동영상 정보 추출

In [15]:
audio_visual_text = '^동영상$|^사진$|^사진 [0-9]{1,2}장$'
mask = df['text'].str.contains(audio_visual_text)
df.loc[mask, 'audio_visual'] = 1
df.loc[~mask, 'audio_visual'] = 0

KeyError: ignored

In [ ]:
# 메시지 건수 조정 (동영상/사진 등을 메시지로 인식한 경우 메시지 건수 0)
df.loc[mask, 'msg_len'] = 0
df.loc[mask, 'msg_word_count'] = 0
df[df['audio_visual']==1]

### 통화 정보 추출하기
- 페이스톡 해요 (Face Call) : 통화 걸기
- 페이스톡 취소 (Cancelled Face Call): 통화 취소
- 페이스톡 부재중 (Missed Video Call): 상대방이 걸어왔지만 내가 응답 없었을 때
- 페이스톡 응답없음 (No Answer on Video Call): 내가 걸었지만 상대방이 응답 없었을 때
- 페이스톡 8:02 (Video Call 8:02): 10분 이내 통화
- 페이스톡 26:25 (Video Call 26:25): 10분 이상 통화
- 페이스톡 1:30:19 (Video Call 1:30:19): 1시간 이상 통화

In [ ]:
df[df['text'].str.contains('보이스톡|페이스톡')]

In [ ]:
def get_call_length(call_df):

    call_mask = call_df.text.str.contains('해요|취소|부재중|응답없음')  # 0
    call_df = call_df.text.str.extract('([0-9]{1,2}:)*([0-9]{1,2}):([0-9]{2})')  # 1
    call_df[0] = call_df[0].str.replace(':', '')  # 2
    call_df[0] = call_df[0].fillna(0)  # 3
    call_df = call_df.astype(float)  # 4
    call_df['call_len'] = call_df[0] * 3660 + call_df[1] * 60 + call_df[2]  # 5
    call_df.loc[call_mask, 'call_len'] = 0  # 6

    return call_df[['call_len']]  # 7

call_col_dict = {
    '페이스톡|Video Call': 'facetalk',
    '보이스톡|Voice Call': 'voicetalk',
}


for call in call_col_dict:
    mask = df['text'].str.contains(call)
    call_df = get_call_length(df[mask])
    call_df = call_df.rename(columns={'call_len': call_col_dict[call]})
    call_df.head()

    df = pd.concat([df, call_df], axis=1)

In [ ]:
mask = (df['voicetalk'] >= 0) | (df['facetalk'] >= 0)
df.loc[mask, 'msg_len'] = 0
df.loc[mask, 'msg_word_count'] = 0

In [ ]:
df[df['text'].str.contains('보이스톡|페이스톡')]

### 비언어 표현 추출하기(이모티콘 등)

| 비언어 표현 | 텍스트 데이터 예시 | 정규표현식 |
|--|--|--|
|웃음, 울음 (ㅋㅋ, ㅎㅎ)  |ㅋㅋ, ㅋㅋㅋㅋ, ㅎㅎㅎㅎㅎ  |[ㅋㅎㅠㅜ]+  |
|느낌표 (!) , 물음표 (?)  |!, !!!!!!!, !?!?, ?????  |[!?~]+  |
| 쉼표(,), 점(.) | ……. ,,,,,, | [,.]{2,} |
| 이모티콘 타입 1 (입 중심)  | :) :D | 	[;:]{1}-?[)(DPpboOX] |
 |이모티콘 타입 1 (눈 중심)  | ^^, ㅜㅜ | ([>ㅜㅠㅡ@\^-+][ㅁㅇ0oO._-] *[\^ㅜㅠㅡ@<-+<];) |
| 이모티콘 타입 2 | 😃😆😍❤ | 정규표현식 대신 emoji 라이브러리 사용 |
| 이모티콘 타입 3 | (반함) (굿) (찡긋) (이모티콘 |  ((.*?)+) |

In [ ]:
df

In [ ]:
# 모두 포함하는 정규 표현식
# (\(.+?\)) | ([ㅋㅎㅠㅜ!?~]+) | [,.]{2,} | ([;:]{1}[\^\'-]?[)(DPpboOX]) | ([>ㅜㅠㅡ@+\^][ㅁㅇ0oO\._\-]*[\^ㅜㅠㅡ@+<];*)

In [ ]:
# 이모지 라이브러리 설치
!pip install emoji

In [ ]:
import emoji

def extract_emojis(text):
    emoji_list = list()
    for c in text:
#         if c in emoji.UNICODE_EMOJI['en']:
        if c in emoji.EMOJI_DATA:
            emoji_list.append(c)

    return emoji_list

mimetic= "[ㅋㅎㅠㅜ!?~]+"
punctuations = "[,.]{2,}"
emo_type1_facial1 = "[;:]{1}[\^\'-]?[)(DPpboOX]"
emo_type1_facial2 = "[>ㅜㅠㅡ@\^][ㅁㅇ0oO\._\-]*[\^ㅜㅠㅡ@<];*"
emo_type3 = "\(.+?\)"

# 1개ㅔ 이상의 비언어 표현을 가지고 있는 데이터 출력하기
nonverbal_list = [mimetic, punctuations, emo_type1_facial1, emo_type1_facial2, emo_type3]

df['nonverbal'] = df['text'].str.findall('|'.join(nonverbal_list)) + df['text'].map(extract_emojis)
df['nonverbal_count'] = df['nonverbal'].apply(len)
df.loc[df['nonverbal_count'] > 0]


#### 이모티콘 사용

In [ ]:
# 이모티콘 사용한 레코드 변경
df.loc[df['text'] == '이모티콘', 'nonverbal'] = '[emoticon]'
df.loc[df['text'] == '이모티콘', 'nonverbal_count'] = 1
df.loc[df['text'] == '이모티콘', 'msg_len'] = 0
df.loc[df['text'] == '이모티콘', 'msg_word_count'] = 0
df.loc[df['text'] == '이모티콘']

In [ ]:
df.loc[df['text'] == '이모티콘'].count()

### URL 추출하기

In [ ]:
# URL 추출 라이브러리 설치하기
!pip install urlextract

In [ ]:
# URL 추출 예제
from urlextract import URLExtract

text = """Google Colab: https://colab.research.google.com.
       Introduction for google colab: https://colab.research.google.com/notebooks/intro.ipynb"""

extractor = URLExtract()
urls = extractor.find_urls(text)

for url in urls:
    print(url)

In [ ]:
from urlextract import URLExtract


extractor = URLExtract()

df['url'] = df['text'].apply(extractor.find_urls)


df['url_count'] = df['url'].apply(len)
url_df = df.loc[df['url_count'] > 0]
url_df[['date_time','user_name','url','url_count']]


### 특정 조건의 컬럼 값 변경

In [ ]:
df.loc[df['user_name'] == 'TMD-최수경-샘', 'user_name'] = '최수경'

### 데이터 저장하기

In [ ]:
fname = 'data/KakaoTalkChats_데이터분석_결과.csv'
df.to_csv(fname)

In [ ]:
df = pd.read_csv(fname, encoding='utf-8')
df

----------------------------------------

## 4. (기술통계)데이터 시각화
- https://wikidocs.net/162798

### 일자별 선/막대 그래프 (plotly)

In [ ]:
import plotly.express as px

# 선그래프
fig = px.line(df, x='date', title='카카오톡 일자별 메시지')
fig

In [ ]:
# 막대 그래프
fig = px.bar(df, x='date', title='카카오톡 일자별 메시지')
fig

In [ ]:
# 점그래프
fig = px.scatter(df, x='date', title='카카오톡 일자별 메시지')
fig

### 사용자별 메시지 건수

In [ ]:
# 유니크한 사용자
df['user_name'].unique()

In [ ]:
# 사용자별 메시지 건수(default  파레토(Pareto) 분포: 내림차순 정렬)
df['user_name'].value_counts()

In [ ]:
# 사용자별 메시지 건수
fig = px.bar(df['user_name'].value_counts())
fig.show()

### 요일별 / 시간별 메시지 건수

In [ ]:
# 요일별 메시지 건수
df['weekday'].value_counts()

In [ ]:
# 시간별 메시지 건수
df['hour'].value_counts()

In [ ]:
# 시간별 메시지 건수(인덱스 순서대로 정렬)
df['hour'].value_counts().sort_index()

In [ ]:
# 시간별 메시지 건수
fig = px.bar(df['hour'].value_counts().sort_index())
fig.show()

### 일자별 히스토그램 : 슬라이더 그래프

In [ ]:
# 일자별 히스토그램
fig = px.histogram(df, x='date')
fig.show()

In [ ]:
# 슬라이더 그래프 생성: 확대해서 보여준다.
fig = px.histogram(df, x='date')

fig.update_xaxes(
    rangeslider_visible=True,
)
fig.show()

In [ ]:
# 사용자별 히스토그램
fig = px.histogram(df, x='date', color='user_name')
fig.show()

------------

THE END